In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
from config import gkey, census_key, yelp_key
from pprint import pprint
import requests

# Census API Key
c = Census(census_key, year=2013)

In [2]:
farm_markets_clean = pd.read_csv("../CleanFM.csv")
farm_markets_clean.head()

,Unnamed: 0,MarketName,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,...,updateTime,Region,Median Age,Population,Poverty Count,Household Income,Per Capita Income,Yelp Results,Yelp Price,Yelp Rating
0,0,Caledonia Farmers Market Association - Danville,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,...,6/20/2017 10:43:57 PM,Northeast,51.2,2011.0,186.0,52292.0,29717.0,NaN,NaN,NaN
1,1,Barre Farmers Market,Barre,Washington,Vermont,05641,05/31/2017 to 10/04/2017,Wed: 4:00 PM-7:30 PM;,NaN,NaN,...,7/4/2017 10:33:01 AM,Northeast,42.6,17298.0,1848.0,49077.0,27985.0,NaN,NaN,NaN
2,2,Bethel Farmers Market,Bethel,Windsor,Vermont,05089,NaN,NaN,NaN,NaN,...,2009,Northeast,46.2,4974.0,565.0,46097.0,27084.0,NaN,NaN,NaN
3,3,Bradford Farmers Market,Bradford,Orange,Vermont,05033,05/18/2014 to 10/19/2014,Sun: 10:00 AM-2:00 PM;,NaN,NaN,...,4/9/2014 10:03:26 AM,Northeast,40.3,2729.0,547.0,43021.0,23173.0,NaN,NaN,NaN
4,4,Brandon Farmers Market,Brandon,Rutland,Vermont,05733,05/22/2015 to 10/09/2015,Fri: 9:00 AM-2:00 PM;,NaN,NaN,...,4/23/2015 3:34:19 PM,Northeast,47.7,5799.0,580.0,53504.0,26931.0,NaN,NaN,NaN


In [3]:
# Make Yelp API calls

search_url = "https://api.yelp.com/v3/businesses/search?"
headers = {"Authorization":"Bearer " + yelp_key}


# change these variables to fit your current API call
index_lower_limit = 0
index_upper_limit = 3000

if index_lower_limit == 0:
    farm_markets_clean["Yelp Results"] = ""
    farm_markets_clean["Yelp Price"] = ""
    farm_markets_clean["Yelp Rating"] = ""

params = {
    "radius":2000,
    "categories":"farmersmarket",
    "limit":5
}

# Yelp Business Search: obtain business ID, price level, and rating
# NOTE: Limit of 5000 requests per day. Divide by 3 to run calls

for index, row in farm_markets_clean.iterrows():
    if (index >= index_lower_limit) and (index <= index_upper_limit):
        params["term"] = row["MarketName"]
        
        # Could search by latitude/longitude or zip code
        # num farmers markets returned from search of 500 with lat/long: 82
        # num farmers markets returned from search of 500 with zip: 55
        # --> search with lat/long
        params["latitude"] = row["Latitude"]
        params["longitude"] = row["Longitude"]

        response = requests.get(search_url, params=params, headers=headers).json()

        try:
            result = response["businesses"][0]
            farm_markets_clean.loc[index, "Yelp Results"] = response["businesses"]
            farm_markets_clean.loc[index, "Yelp Rating"] = result["rating"]
            farm_markets_clean.loc[index, "Yelp Price"] = result["price"]
        except:
            print(f"No nearby market for {row['MarketName']}")

No nearby market for Walpole Farmers Market - Winter
No nearby market for Warner Area Farmers Market
No nearby market for We Are One Farmers Market
No nearby market for Weare Farmers Market 
No nearby market for Weare Farmers Market (Summer)
No nearby market for Weare Farmers Market (winter)
No nearby market for Wentworth Greenhouse Farmers' Market
No nearby market for Whitefield Farmers' Market
No nearby market for Wilmot Farmers Market
No nearby market for Wolfeboro Area Farmers' Market
No nearby market for Basin Farmers Market
No nearby market for Big Horn Basin Farmers Market Cody
No nearby market for Big Horn Basin Farmers Market Powell
No nearby market for Buffalo Main Street Market
No nearby market for Cheyenne Farmers Market
No nearby market for Cheyenne Tuesday Farmers' Market
No nearby market for Cheyenne Winter Farmers' Market
No nearby market for Craft Market
No nearby market for Downtown Laramie Farmer's market
No nearby market for Dubois Farmers' Market
No nearby market f

In [5]:
valid_yelp = farm_markets_clean[farm_markets_clean["Yelp Rating"].isnull() == False]

# Verify that market from Yelp matches market from CSV file
for index, row in valid_yelp.iterrows():
    if index >= index_lower_limit and index <= index_upper_limit:
        # convert to lowercase and strip apostrophes
        csv_name = row["MarketName"].lower().replace("'","")
        yelp_name = row["Yelp Results"]["name"].lower().replace("'","")

        csv_name = csv_name.replace("farmers market", "")
        yelp_name = yelp_name.replace("farmers market", "")

        csv_words = csv_name.split(" ")
        yelp_words = yelp_name.split(" ")

    # To make sure that all farmers market names are reasonably equal:
        # strip apostrophes
        # convert to lowercase
        # remove "farmers market"
        # test if new strings are equal OR if the csv name contains word in the yelp name

        overlap_words = False

        for word in csv_words:
            if (word.isalnum()) and (word in yelp_words):
                overlap_words = True

        if not((csv_name == yelp_name) or overlap_words):
            farm_markets_clean.loc[index, "Yelp Results"] = "NOT EQUAL"

In [6]:
# Print number of valid markets returned

farm_markets_clean[farm_markets_clean["Yelp Rating"].isnull() == False].shape

(1384, 61)

In [7]:
farm_markets_clean.to_csv("../CleanFM.csv", index=False)